# Return values

This notebook focuses on return information from fastapi handlers.

Container for the examples in this page. Check [run application](../fastapi.ipynb#run-application) for more details about image that is used for this container.

In [22]:
import requests

!docker run --rm -itd\
    --name test_container\
    -v ./return_values_files/app.py:/app.py\
    -p 8000:8000 \
    fastapi_experiment \
    uvicorn --host 0.0.0.0 --reload app:app

6bec5af2cde3018c3230c2fd4e2e7ba03327f9877655f7e7c618feecaea3e876


**Note** Don't forget to stop the container.

In [14]:
!docker stop test_container &>/dev/null

## Single value types

Consider the case when you are returning types that contain a value like `int`, `float`, `str` and so on.

### Unannotated


**It does not have to be declared**. So in the following example the output data type is not specified and the query is executed without problems.

But you will always have string as the output type.

In [19]:
%%writefile return_values_files/app.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/divide")
def divide(a : int, b : int):
    return a/b

Overwriting return_values_files/app.py


In [10]:
response = requests.get("http://localhost:8000/divide?a=10&b=2")
response.text

'5.0'

### Annotated

**But if you have annotated a type, you must follow it.** 

In the following example, the `GET` response function is configured to use `int` as output. 

- The first `request` called in such a way as to return `float` - that's why you got the error;
- The result of the second `request` can be interpreted as `int`, so all is well.

In [11]:
%%writefile return_values_files/app.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/divide")
def divide(a : int, b : int) -> int:
    return a/b

Overwriting return_values_files/app.py


In [12]:
response = requests.get("http://localhost:8000/divide?a=1&b=2")
response.text

'Internal Server Error'

In [13]:
response = requests.get("http://localhost:8000/divide?a=4&b=2")
response.text

'2'

## `dict`, `list` -> JSON

If you return a list or dictionary from your handler, it will be converted to the corresponding json file in the api output.

The following cell defines an application that has both handlers that return dictionary and list. 

In [15]:
%%writefile return_values_files/app.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/dict")
def return_dict():
    return {
        "key1" : "value1",
        "key2" : "value2",
        6 : 34
    }

@app.get("/list")
def return_json():
    return [1, 2, 3, "hello", True]

Overwriting return_values_files/app.py


Now let's look at the results for both handlers.

In [4]:
!curl localhost:8000/dict
!curl localhost:8000/list

{"key1":"value1","key2":"value2","6":34}

[1,2,3,"hello",true]

**Note** that all keys in the dictionary have been converted to strings. So originally 6 was converted to "6".

## Pydantic models

You can return instances of the pydantic models - it will convert the output to the corresponding Json file.

The following example shows an application that uses the pydantic model to define what a program must return.

In [23]:
%%writefile return_values_files/app.py
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Output(BaseModel):
    a : int
    b : str

@app.get("/")
def return_dict() -> Output:
    return Output(a=10, b="string value")

Overwriting return_values_files/app.py


In [24]:
response = requests.get("http://localhost:8000/")
response.text

'{"a":10,"b":"string value"}'